In [ ]:
# Colab-friendly full Dask pipeline for Distributed Stock Market Trend Analysis
# - Installs deps
# - Downloads stock CSVs (yfinance)
# - Starts Dask distributed Client
# - Loads CSVs using dask.dataframe
# - Computes Daily_Return & Price_Change via map_partitions
# - Persists intermediate results
# - Computes avg returns, volatility, avg volume, correlation matrix
# - Validates using Pandas on a small sample
# - Visualizes results with Plotly
# - Prints timing and Dask dashboard link

# -------------------------
# 1) Install required packages (Colab)
# -------------------------
!pip install --quiet dask[complete] distributed pandas yfinance plotly

# -------------------------
# 2) Imports & configuration
# -------------------------
import os
import time
from datetime import datetime
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import plotly.express as px
import plotly.graph_objects as go

# Configuration (edit if you want)
STOCKS = ["AAPL","TSLA","MSFT","NVDA","AMZN","GOOGL","META","JPM"]
START_DATE = "2015-10-25"
END_DATE   = "2025-10-22"
DATA_DIR = "/content/data"

os.makedirs(DATA_DIR, exist_ok=True)

# -------------------------
# 3) Download historical data using yfinance and save CSVs
# -------------------------
import yfinance as yf
print("📥 Downloading stock data...")
start_time = time.perf_counter()
for ticker in STOCKS:
    out_path = os.path.join(DATA_DIR, f"{ticker}.csv")
    if not os.path.exists(out_path):
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, auto_adjust=False)
        if df.empty:
            print(f"⚠️ No data for {ticker}")
            continue
        # Normalise columns, add ticker column and Date as column
        df = df.reset_index()
        df.rename(columns={"Adj Close":"Adj_Close"}, inplace=True)
        df["Ticker"] = ticker
        df.to_csv(out_path, index=False)
        print(f"   Downloaded {ticker} -> {out_path} : {len(df)} rows")
    else:
        print(f"   Exists {ticker}, skipping download")
download_time = time.perf_counter() - start_time
print(f"✅ Download finished in {download_time:.2f}s")

# -------------------------
# 4) Start local Dask cluster and client
# -------------------------
print("\n🚀 Starting local Dask cluster...")
cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit="2GB")
client = Client(cluster)
# Show dashboard link (Colab may block direct, still printed)
print("Dask dashboard:", client.dashboard_link)
print("Cluster info:", client.cluster)

# -------------------------
# 5) Load CSVs with Dask
# -------------------------
csv_glob = os.path.join(DATA_DIR, "*.csv")
print("\n📂 Loading data from:", csv_glob)
t0 = time.perf_counter()
ddf = dd.read_csv(csv_glob, assume_missing=True, dtype={'Volume':'float64'})
# ensure Date col parsed as datetime (delayed parse)
ddf['Date'] = dd.to_datetime(ddf['Date'])
print("   Columns:", list(ddf.columns))
print("   Partitions:", ddf.npartitions)
load_time = time.perf_counter() - t0
print(f"✅ Lazy load prepared in {load_time:.3f}s (no compute yet)")

# -------------------------
# 6) Feature engineering: compute daily returns + price change safely
#    We must compute pct_change per Ticker in chronological order.
#    Use map_partitions which runs a Pandas function over each partition.
# -------------------------
def compute_features(pdf):
    # ensure Date sorted inside each partition; this function runs per-partition
    if "Date" in pdf.columns:
        pdf = pdf.sort_values("Date")
    # create daily return and price change (pct_change works per contiguous rows)
    # Note: pct_change across partition boundary may be undefined; acceptable for this beginner pipeline.
    pdf["Daily_Return"] = pdf["Close"].pct_change() * 100.0   # percent
    pdf["Price_Change"] = pdf["Close"] - pdf["Open"]
    # Fill first-day NaNs with 0 to avoid propagation
    pdf["Daily_Return"] = pdf["Daily_Return"].fillna(0.0)
    pdf["Price_Change"] = pdf["Price_Change"].fillna(0.0)
    return pdf

print("\n🔧 Computing features (map_partitions)...")
t0 = time.perf_counter()
# Corrected meta argument to specify dtypes using pandas Series
ddf = ddf.map_partitions(compute_features, meta=ddf._meta.assign(Daily_Return=pd.Series([], dtype='float64'), Price_Change=pd.Series([], dtype='float64')))
# Persist to keep the computed representation in cluster memory (speeds repeated ops)
ddf = ddf.persist()
client.wait_for_workers(1)  # ensure worker availability
# Force a small compute to materialize persisted representation
ddf.head(3)
feature_time = time.perf_counter() - t0
print(f"✅ Feature engineering submitted and persisted in {feature_time:.3f}s")

# -------------------------
# 7) Aggregations: mean return, volatility, avg volume
# -------------------------
print("\n📊 Running aggregated computations (this will compute)")
t0 = time.perf_counter()

# Average daily return per Ticker
avg_return = ddf.groupby("Ticker")["Daily_Return"].mean().compute().rename("Avg_Return_%")
# Volatility (std dev of daily returns)
volatility = ddf.groupby("Ticker")["Daily_Return"].std().compute().rename("Volatility")
# Average volume
avg_volume = ddf.groupby("Ticker")["Volume"].mean().compute().rename("Avg_Volume")

# Combine into a summary DataFrame
summary = pd.concat([avg_return, volatility, avg_volume], axis=1).sort_values("Avg_Return_%", ascending=False)
agg_time = time.perf_counter() - t0
print(f"✅ Aggregations completed in {agg_time:.2f}s")

print("\nTop performing stocks by Avg_Return_%:")
print(summary.head(10).round(4))

# -------------------------
# -------------------------
# 8) Volatility time series for plotting: compute rolling std per ticker
# -------------------------
print("\n📈 Computing volatility time-series for plotting (per ticker via compute)...")
t0 = time.perf_counter()

# Select tickers to plot (top 4 by avg return)
plot_tickers = list(summary.head(4).index)

# Extract Date, Ticker, Daily_Return into pandas (only for selected tickers)
subset = ddf[ddf["Ticker"].isin(plot_tickers)][["Date","Ticker","Daily_Return","Close"]].compute()

# ✅ FIX: Handle timezone-aware datetimes
subset["Date"] = pd.to_datetime(subset["Date"], utc=True).dt.tz_convert(None)

# Compute rolling volatility (30-day window) per ticker
subset = subset.sort_values(["Ticker","Date"])
subset["Rolling_Volatility_30d"] = subset.groupby("Ticker")["Daily_Return"].transform(lambda s: s.rolling(window=30, min_periods=5).std())
rolling_time = time.perf_counter() - t0
print(f"✅ Time series prepared in {rolling_time:.2f}s")


# -------------------------
# 9) Correlation matrix of closing prices across all tickers
#    Build a pivot table Date x Ticker of Close, then compute correlation
# -------------------------
print("\n🔗 Building correlation matrix of Close prices...")
t0 = time.perf_counter()
close_df = ddf[["Date","Ticker","Close"]].compute()
close_df["Date"] = pd.to_datetime(close_df["Date"])
pivot = close_df.pivot_table(index="Date", columns="Ticker", values="Close")
# Optionally fill small gaps by forward fill
pivot_ffill = pivot.fillna(method="ffill").fillna(method="bfill")
corr = pivot_ffill.corr()
corr_time = time.perf_counter() - t0
print(f"✅ Correlation matrix computed in {corr_time:.2f}s")

# -------------------------
# 10) Validation: pick one ticker CSV, compute avg return in Pandas and compare
# -------------------------
print("\n🔬 Validation (Pandas single-file check):")
t0 = time.perf_counter()
sample_ticker = STOCKS[0]
sample_path = os.path.join(DATA_DIR, f"{sample_ticker}.csv")
pdf = pd.read_csv(sample_path, parse_dates=["Date"])
pdf = pdf.sort_values("Date")
pdf["Daily_Return"] = pdf["Close"].pct_change().fillna(0.0) * 100.0
pandas_avg = pdf["Daily_Return"].mean()
dask_avg = avg_return.loc[sample_ticker]
print(f"   {sample_ticker} - Pandas Avg_Return_%: {pandas_avg:.6f}, Dask Avg_Return_%: {dask_avg:.6f}")
print(f"Validation delta: {abs(pandas_avg - dask_avg):.8f}")
val_time = time.perf_counter() - t0

# -------------------------
# 11) Visualizations (Plotly)
# -------------------------
print("\n📊 Plotting results with Plotly...")

# Top 5 performing stocks bar chart
top5 = summary.head(5).reset_index()
fig1 = px.bar(top5, x="Ticker", y="Avg_Return_%", title="Top 5 Performing Stocks (Avg Daily Return %)")
fig1.update_layout(yaxis_title="Avg Daily Return (%)")
fig1.show()

# Volatility trends for top tickers
fig2 = go.Figure()
for t in plot_tickers:
    df_t = subset[subset["Ticker"] == t]
    fig2.add_trace(go.Scatter(x=df_t["Date"], y=df_t["Rolling_Volatility_30d"], mode="lines", name=f"{t}"))
fig2.update_layout(title="30-Day Rolling Volatility (Daily Return %) for Top Tickers", yaxis_title="Rolling Volatility (%)", xaxis_title="Date")
fig2.show()

# Correlation heatmap
fig3 = px.imshow(corr, text_auto=True, title="Correlation Matrix of Close Prices")
fig3.update_layout(width=800, height=700)
fig3.show()

# -------------------------
# 12) Summary prints and timings
# -------------------------
total_time = download_time + load_time + feature_time + agg_time + rolling_time + corr_time
print("\n======================== SUMMARY ========================")
print(f"Stocks: {STOCKS}")
print(f"Date range: {START_DATE} to {END_DATE}")
print(f"Data directory: {DATA_DIR}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Top 5 stocks by Avg_Return_%:\n{top5.set_index('Ticker')['Avg_Return_%'].round(4)}")
print("\nTimings (s):")
print(f"  Download: {download_time:.2f}")
print(f"  Lazy load prepare: {load_time:.3f}")
print(f"  Feature engineering submit/persist: {feature_time:.3f}")
print(f"  Aggregations: {agg_time:.2f}")
print(f"  Rolling series prep: {rolling_time:.2f}")
print(f"  Correlation: {corr_time:.2f}")
print(f"  Validation (pandas sample): {val_time:.4f}")
print(f"Estimated total pipeline time (approx): {total_time:.2f}s")
print("Dask dashboard:", client.dashboard_link)
print("========================================================")

# -------------------------
# 13) Clean up (if you want to shut down)
# -------------------------
# client.close()
# cluster.close()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39879
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:42261/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41653'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45003'


📥 Downloading stock data...
   Exists AAPL, skipping download
   Exists TSLA, skipping download
   Exists MSFT, skipping download
   Exists NVDA, skipping download
   Exists AMZN, skipping download
   Exists GOOGL, skipping download
   Exists META, skipping download
   Exists JPM, skipping download
✅ Download finished in 0.00s

🚀 Starting local Dask cluster...


INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46805'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41259'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:35779 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35779
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39350
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:39881 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39881
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39352
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:40529 name: 2
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40529
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39362
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:34671 name: 3
INFO:distributed.scheduler:Starting worker compute stream, tcp

Dask dashboard: http://127.0.0.1:42261/status
Cluster info: LocalCluster(ff529bbc, 'tcp://127.0.0.1:39879', workers=4, threads=8, memory=7.45 GiB)

📂 Loading data from: /content/data/*.csv
   Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']
   Partitions: 8
✅ Lazy load prepared in 0.133s (no compute yet)

🔧 Computing features (map_partitions)...
✅ Feature engineering submitted and persisted in 28.728s

📊 Running aggregated computations (this will compute)
✅ Aggregations completed in 1.80s

Top performing stocks by Avg_Return_%:
        Avg_Return_%  Volatility    Avg_Volume
Ticker                                        
NVDA          0.2706      3.1450  4.610751e+08
TSLA          0.2056      3.7341  1.150373e+08
MSFT          0.1097      1.6845  2.772706e+07
AAPL          0.1085      1.8388  1.041287e+08
META          0.1076      2.4169  2.180607e+07
AMZN          0.0995      2.0589  7.374626e+07
GOOGL         0.0933      1.8032  3.4481


======================== SUMMARY ========================
Stocks: ['AAPL', 'TSLA', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META', 'JPM']
Date range: 2015-10-25 to 2025-10-22
Data directory: /content/data
Workers: 4
Top 5 stocks by Avg_Return_%:
Ticker
NVDA    0.2706
TSLA    0.2056
MSFT    0.1097
AAPL    0.1085
META    0.1076
Name: Avg_Return_%, dtype: float64

Timings (s):
  Download: 0.00
  Lazy load prepare: 0.133
  Feature engineering submit/persist: 28.728
  Aggregations: 1.80
  Rolling series prep: 1.05
  Correlation: 1.02
  Validation (pandas sample): 0.0429
Estimated total pipeline time (approx): 32.73s
Dask dashboard: http://127.0.0.1:42261/status
